In [ ]:
%pip install PyPDF2==3.0.1 unstructured==0.8.3

In [ ]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb

In [ ]:
!pip install kaggle

In [ ]:
import google.generativeai as genai
import PyPDF2
from unstructured.partition.pdf import partition_pdf
from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.colab import userdata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [ ]:
import nltk

# Download the 'punkt_tab' dataset
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('punkt')


# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_file_path = '/content/drive/MyDrive/CSV_Files/mastercard_rules.pdf'
# Extract text from the PDF
pdf_text = ""
with open(pdf_file_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_text += page.extract_text()

# Option 2: Chunk by paragraphs (choose one based on your needs)
paragraphs = pdf_text.split('\n\n')
documents = paragraphs

elements = partition_pdf(filename=pdf_file_path, strategy="fast")
documents = [str(el) for el in elements]

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import chromadb

DB_NAME = "pdf_db"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

#db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

# Filter out empty documents before adding to ChromaDB
filtered_documents = [doc for doc in documents if doc.strip()]  # Remove empty or whitespace-only documents
filtered_ids = [str(i) for i, doc in enumerate(documents) if doc.strip()]  # Generate corresponding IDs

db.add(documents=filtered_documents, ids=filtered_ids)

Streaming output truncated to the last 5000 lines.


In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "What are your responsibilities to cardholders?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

Issuer Responsibilities to Cardholders

In [ ]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot named Alex that answers questions using text from the reference passage included below. Your goal is to
provide accurate, helpful, and relevant answers.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
Be sure to break down complicated concepts and strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.
Also do not give any indication that this information is from a passage/text. If you are not sure that you have the specific answer to question being asked, then give information based on a passage that contains key words based on the question being asked. Also always summarize answers longer than 100 words
unless the person ask for specifics.

## Example Interaction:
**Question:** What are the late payment fees?
**Answer:** Prior to Card usage, the Issuer is obligated to inform the Cardholder that the Card is accepted wherever the designated Marks are displayed.
This disclosure must also include the cost of the Card,
any applicable charges for services provided through the Card or
auxiliary account charges (such as Account access fees, cash advance fees,
ATM usage fees, late payment fees, and interest rates),
the method used to calculate exchange rates,
and a clear notification that exchange rates are subject to
fluctuation and may vary between the Transaction date and the billing date on the Cardholder's Account.

**Question:** How are overpayments handled?
In this example provide what ever information you have on overpayments.

**Question:** What does MasterCard do?
In this example provide what ever information you have on MasterCards goals.




##
QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot named Alex that answers questions using text from the reference passage included below. Your goal is to 
provide accurate, helpful, and relevant answers. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
Be sure to break down complicated concepts and strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it. 
Also do not give any indication that this information is from a passage/text. If you are not sure that you have the specific answer to question being asked, then give information based on a passage that contains key words based on the question being asked. Also always summarize answers longer than 100 words 
unless the person ask for specifics.

## Example Interaction:
**Question:** What are the late payment fees?
**Answer:** Prior to Card usage, the Issuer is obligated to inform the Cardholder that the Card is accepted wherever the desi

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

Before you start using your card, the issuer must tell you where it's accepted (look for the designated marks!),  the card's cost, any fees (like late payment fees or cash advance fees), how exchange rates are calculated, and that those rates can change between the transaction and your billing date.
